In [1]:
import pandas as pd
import numpy as np
import gensim.downloader as api
from gensim import models
from gensim.models import KeyedVectors

In [2]:
# Importing previously stored dataset of 

data = pd.read_csv('amazon_reviews_us_Kitchen_v1_00.tsv', error_bad_lines=False, warn_bad_lines=False, sep='\t')
df = data[['review_body', 'star_rating']]
df['rating'] = df['star_rating'].apply(lambda x: 1 if x > 3 else (2 if x<3 else 3))
#df['rating'] = df['star_rating'].apply(lambda x: 3 if x==3 else df['rating'])
df.head()

/Users/sidmitta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,review_body,star_rating,rating
0,Beautiful. Looks great on counter.,5.0,1
1,I personally have 5 days sets and have also bo...,5.0,1
2,Fabulous and worth every penny. Used for clean...,5.0,1
3,A must if you love garlic on tomato marinara s...,5.0,1
4,Worth every penny! Buy one now and be a pizza ...,5.0,1


In [3]:
# Loading inbuilt w2v model
wv = api.load('word2vec-google-news-300')

In [4]:
# Saving and loading inbuilt model from directory to avoid download everytime
df1 = df.query(' star_rating == 1').sample(n=50000)
df2 = df.query(' star_rating == 2').sample(n=50000)
df3 = df.query(' star_rating == 3').sample(n=50000)
df4 = df.query(' star_rating == 4').sample(n=50000)
df5 = df.query(' star_rating == 5').sample(n=50000)
frames = [df1, df2, df3, df4, df5]
df = pd.concat(frames)
df = df.sample(frac=1)
wv.save("google_news.d2v")
#wv = KeyedVectors.load("google_news.d2v")

In [5]:
# Taking 2 sample rows
df.head()
wv_sample = df['review_body'].head(2)

In [6]:
# Obtaining vectors for sample sentences using word2vec-google-news-300 model

try:
  for sentence in wv_sample:
    words = sentence.split()
    for word in words:
      v = wv[word]
      print("Word: " + word)
      #print(v)
    
except KeyError:
    print("The word " + word + " does not appear in this model")

Word: Pot
Word: dies
Word: not
Word: age
Word: well
Word: on
The word a does not appear in this model


In [7]:
wv["king"]
wv["man"]
va=wv["king"]-wv["man"]+wv["woman"]
vb=wv["queen"]
va1=wv["nice"]
vb1=wv["good"]
print(np.dot(va,vb)/(np.linalg.norm(va)* np.linalg.norm(vb)))
print(np.dot(va1,vb1)/(np.linalg.norm(va1)* np.linalg.norm(vb1)))


0.73005176
0.68360925


In [8]:
wv.vectors.shape

(3000000, 300)

In [9]:
# Splitting reviews into a list of words - tokenizing
import re
df["review_body"] = df['review_body'].str.lower()
df['review_body'] = df['review_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, ' ', raw_html)
  return cleantext

df['review_body'] = df['review_body'].apply(lambda x: cleanhtml(x))
df['review_body'] = df['review_body'].str.replace('[^ a-zA-Z]', '')

def contractionfunction(phrase):
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

df['review_body'] = df.review_body.apply(lambda row: contractionfunction(row))

from nltk.corpus import stopwords
stop = stopwords.words('english')
df['review_body'] = df['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

df['review_body'] = df.review_body.apply(lambda x: lemmatize_text(x))

In [11]:
df_testing=df[["review_body", "rating"]]
df_testing["review_body"] = df_testing['review_body'].str.replace('[^\w\s]','')
df_testing['review_body']=df_testing['review_body'].apply(lambda x: re.sub("\d+","", x))
df_testing['review_body']=df_testing['review_body'].apply(lambda x: x.strip())
df_testing['review_body'] = df_testing['review_body'].replace('\s+', ' ', regex=True)

/Users/sidmitta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/sidmitta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sidmitta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [164]:
sentences = []
for i in df_testing['review_body']:
  if type(i).__name__ == "str":
    sentences.append(i.split())
mymodel = models.Word2Vec(sentences = sentences, min_count = 10, size = 300, window = 11, workers = 10)

In [165]:
mymodel.save("mymodel.d2v")
#mymodel = KeyedVectors.load("mymodel.d2v")

In [166]:
mymodel.wv.vectors.shape

(14903, 300)

In [15]:
# Comparing similarity (top 5) for 2 sample words from dataset, for inbuilt and custom models respectively

print("Similarity score for idea from inbult model: ", wv.most_similar('bad', topn = 5))
print("Similarity score for idea from custom model: ", mymodel.wv.most_similar('bad', topn = 5))

print("Similarity score for perfect from inbult model: ", wv.most_similar('amazing', topn = 5))
print("Similarity score for perfect from custom model: ", mymodel.wv.most_similar('amazing', topn = 5))

Similarity score for idea from inbult model:  [('good', 0.7190051078796387), ('terrible', 0.6828612089157104), ('horrible', 0.6702598333358765), ('Bad', 0.669891893863678), ('lousy', 0.6647640466690063)]
Similarity score for idea from custom model:  [('good', 0.5042324662208557), ('terrible', 0.5006387233734131), ('awful', 0.477702796459198), ('horrible', 0.44045141339302063), ('poor', 0.43576285243034363)]
Similarity score for perfect from inbult model:  [('incredible', 0.9054000973701477), ('awesome', 0.8282865285873413), ('unbelievable', 0.8201264142990112), ('fantastic', 0.7789871096611023), ('phenomenal', 0.7642048001289368)]
Similarity score for perfect from custom model:  [('awesome', 0.7773792147636414), ('fantastic', 0.7667995691299438), ('wonderful', 0.6966757774353027), ('fabulous', 0.6923918724060059), ('amazed', 0.6404046416282654)]


In [16]:
df_binary = df_testing[df_testing['rating'] !=3]
df_binary.dropna()
df_binary_2=df_binary.copy()

In [17]:
wv_trained = pd.DataFrame()
mymodel_trained = pd.DataFrame()

def gentrain(s,model):
    word_list = train(s,model)
    word_list = np.array(word_list)
    word_list = word_list.mean(axis=0)
    return word_list

def train(s,model):
    words=s.split()
    each_word=[]
    for word in words:
        if word in model:
            each_word.append(model[word])
        else:
            each_word.append(np.zeros(300))
    return each_word

    
wv_trained['review_body']=df_binary['review_body'].apply(lambda row: gentrain(row, wv))
mymodel_trained['review_body']=df_binary['review_body'].apply(lambda row: gentrain(row, mymodel.wv))


/Users/sidmitta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice.
  import sys


In [18]:
def normalize(df):
    n_list=[]
    for i in df:
        i=i.tolist()
        if type(i).__name__=='float':
            i=[0.0 for i in range(300)]
        n_list.append(i)
    y=np.array([np.array(x) for x in n_list])
    return y

In [19]:
wv_trained.shape

(200000, 1)

In [20]:
n_wv_trained=normalize(wv_trained['review_body'])
n_mymodel_trained=normalize(mymodel_trained['review_body'])

In [21]:
df_binary['rating'].shape

(200000,)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

avg_my_model_x_train, avg_my_model_x_test, avg_my_model_y_train, avg_my_model_y_test = train_test_split(n_mymodel_trained, df_binary["rating"],test_size=0.2, random_state=0)
avg_wv_x_train, avg_wv_x_test, avg_wv_y_train, avg_wv_y_test = train_test_split(n_wv_trained, df_binary["rating"],test_size=0.2, random_state=0)

In [23]:
avg_my_model_y_train.shape

(160000,)

In [24]:
#CUSTOM MODEL Accuracy using Perceptron
from sklearn.linear_model import Perceptron
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(avg_my_model_x_train, avg_my_model_y_train)

perceptron_pred_train = clf.predict(avg_my_model_x_train)
perceptron_pred_test = clf.predict(avg_my_model_x_test)

print('%.3f' % accuracy_score(avg_my_model_y_train, perceptron_pred_train))
print('%.3f' % accuracy_score(avg_my_model_y_test, perceptron_pred_test))

0.822
0.821


In [25]:
#WV MODEL Accuracy using Perceptron
from sklearn.linear_model import Perceptron
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
clf1 = Perceptron(tol=1e-3, random_state=0)
clf1.fit(avg_wv_x_train, avg_wv_y_train)

perceptron_pred_train = clf1.predict(avg_wv_x_train)
perceptron_pred_test = clf1.predict(avg_wv_x_test)

print('%.3f' % accuracy_score(avg_wv_y_train, perceptron_pred_train))
print('%.3f' % accuracy_score(avg_wv_y_test, perceptron_pred_test))

0.714
0.716


In [199]:
#TF_IDF splitting
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_testing['review_body'])
train_X, test_X, train_Y, test_Y = train_test_split(X, df_testing['rating'],test_size=0.2, random_state=0)

In [200]:
#TF_IDF accuracy using Perceptron
clf2 = Perceptron(tol=1e-3, random_state=0)
clf2.fit(train_X, train_Y)

perceptron_pred_train = clf2.predict(train_X)
perceptron_pred_test = clf2.predict(test_X)

print('%.3f' % accuracy_score(train_Y,perceptron_pred_train))
print('%.3f' % accuracy_score(test_Y,perceptron_pred_test))

0.772
0.647


In [28]:
#Custom model accuracy using SVM
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(avg_my_model_x_train, avg_my_model_y_train)

SVM_pred_train = clf.predict(avg_my_model_x_train)
SVM_pred_test = clf.predict(avg_my_model_x_test)

print('%.3f' % accuracy_score(avg_my_model_y_train, SVM_pred_train))
print('%.3f' % accuracy_score(avg_my_model_y_test, SVM_pred_test))

0.861
0.860


/Users/sidmitta/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [29]:
#WV model accuracy using SVM
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(avg_wv_x_train, avg_wv_y_train)
SVM_pred_train = clf.predict(avg_wv_x_train)
SVM_pred_test = clf.predict(avg_wv_x_test)

print('%.3f' % accuracy_score(avg_wv_y_train, SVM_pred_train))
print('%.3f' % accuracy_score(avg_wv_y_test, SVM_pred_test))

0.822
0.820


In [30]:
#TF_IDF model accuracy using SVM
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(train_X, train_Y)
SVM_pred_train = clf.predict(train_X)
SVM_pred_test = clf.predict(test_X)

print('%.3f' % accuracy_score(train_Y, SVM_pred_train))
print('%.3f' % accuracy_score(test_Y, SVM_pred_test))

0.801
0.707


In [31]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [32]:
ptdf=df[df['star_rating']!="3"]
msk = np.random.rand(len(ptdf)) < 0.8
pt_train = ptdf[msk]
pt_test = ptdf[~msk]

/Users/sidmitta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [33]:
ptdf.head()

,review_body,star_rating,rating
509400,pot dy age well gas stove surface corrosion st...,2.0,2
1245218,broke first time used snapped half middle piec...,1.0,2
2676882,recieved item plastic handle broken handle mad...,1.0,2
4845252,got coffee maker spent brewed first cup coffee...,2.0,2
3865398,love popcorn popper year later still making ba...,5.0,1


In [197]:
df_binary["rating"]=df_binary["rating"].apply(lambda x: 0 if x==1 else x).any()
df_binary["rating"]=df_binary["rating"].apply(lambda x: 1 if x==2 else x).any()

/Users/sidmitta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/sidmitta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
avg_wv_x_train, avg_wv_x_test, avg_wv_y_train, avg_wv_y_test = train_test_split(n_wv_trained, df_binary["rating"],test_size=0.2, random_state=0)

In [36]:
x_train = torch.tensor(avg_wv_x_train)
y_train = torch.tensor(avg_wv_y_train.astype(np.float32).values, dtype=torch.long)
x_test = torch.tensor(avg_wv_x_test)
y_test = torch.tensor(avg_wv_y_test.astype(np.float32).values, dtype=torch.long)

In [37]:
train_data = []
for i in range(len(x_train)):
    train_data.append([x_train[i], y_train[i]])

In [38]:
test_data = []
for i in range(len(x_test)):
    test_data.append([x_test[i], y_test[i]])

In [39]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data)
#indices = list(range(num_train))
indices = [i for i in range(num_train)]
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [40]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers,)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

In [193]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (512)
        hidden_1 = 50
        hidden_2 = 10
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(300, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(hidden_2, 2)
        
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # flatten image input
        #x = x.view(-1, 300)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x.float()))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        #x = self.softmax(x)
        return(F.log_softmax(x,dim=1))

# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=300, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [194]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [195]:
def train_model():
    # number of epochs to train the model
    n_epochs = 40

    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf # set initial "min" to infinity

    for epoch in range(n_epochs):
        # monitor training loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        for data, target in train_loader:
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.float())
            # calculate the loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update running training loss
            train_loss += loss.item()*data.size(0)

        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation
        for data, target in valid_loader:
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.float())
            # calculate the loss
            loss = criterion(output, target)
            # update running validation loss 
            valid_loss += loss.item()*data.size(0)

        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)

        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch+1, 
            train_loss,
            valid_loss
            ))

        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), 'model.pt')
            valid_loss_min = valid_loss

In [ ]:
train_model()

In [45]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [46]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, 
    num_workers=num_workers)      

In [176]:
def test_model():
    total = len(test_data)
    correct = 0
    with torch.no_grad():
        model.eval()
        for data in test_loader:
            reviews, labels = data
            outputs = model(reviews)
            _,predicted = torch.max(outputs.data,1)
            correct += (predicted == labels).sum().item()


    print('Accuracy of the network on the test images: %d %%' % (
        100 * correct / total))

In [48]:
#WV Model accuracy using Feedforward Neural network for binary classification
test_model()

Accuracy of the network on the test images: 100 %


In [49]:
avg_my_model_x_train, avg_my_model_x_test, avg_my_model_y_train, avg_my_model_y_test = train_test_split(n_mymodel_trained, df_binary["rating"],test_size=0.2, random_state=0)

In [50]:
x_train = torch.tensor(avg_my_model_x_train)
y_train = torch.tensor(avg_my_model_y_train.astype(np.float32).values, dtype=torch.long)
x_test = torch.tensor(avg_my_model_x_test)
y_test = torch.tensor(avg_my_model_y_test.astype(np.float32).values, dtype=torch.long)

In [51]:
train_data = []
for i in range(len(x_train)):
    train_data.append([x_train[i], y_train[i]])
test_data = []
for i in range(len(x_test)):
    test_data.append([x_test[i], y_test[i]])
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data)
#indices = list(range(num_train))
indices = [i for i in range(num_train)]
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [52]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers,)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

In [53]:
train_model()

Epoch: 1 	Training Loss: 0.000130 	Validation Loss: 0.000031
Validation loss decreased (inf --> 0.000031).  Saving model ...
Epoch: 2 	Training Loss: 0.000126 	Validation Loss: 0.000030
Validation loss decreased (0.000031 --> 0.000030).  Saving model ...
Epoch: 3 	Training Loss: 0.000122 	Validation Loss: 0.000029
Validation loss decreased (0.000030 --> 0.000029).  Saving model ...
Epoch: 4 	Training Loss: 0.000118 	Validation Loss: 0.000028
Validation loss decreased (0.000029 --> 0.000028).  Saving model ...
Epoch: 5 	Training Loss: 0.000115 	Validation Loss: 0.000027
Validation loss decreased (0.000028 --> 0.000027).  Saving model ...
Epoch: 6 	Training Loss: 0.000111 	Validation Loss: 0.000027
Validation loss decreased (0.000027 --> 0.000027).  Saving model ...
Epoch: 7 	Training Loss: 0.000108 	Validation Loss: 0.000026
Validation loss decreased (0.000027 --> 0.000026).  Saving model ...
Epoch: 8 	Training Loss: 0.000105 	Validation Loss: 0.000025
Validation loss decreased (0.00002

In [54]:
#Custom model accuracy using Feed forward neural network for binary classification
test_model()

Accuracy of the network on the test images: 100 %


In [55]:
df_ternary=df_testing.copy()

In [56]:
df_ternary.dropna()
df_ternary

,review_body,rating
509400,pot dy age well gas stove surface corrosion st...,2
1245218,broke first time used snapped half middle piec...,2
2676882,recieved item plastic handle broken handle mad...,2
4845252,got coffee maker spent brewed first cup coffee...,2
3865398,love popcorn popper year later still making ba...,1
...,...,...
3398974,utensil came handy made well sturdy practical ...,1
4204428,shipped fast exactly looking plastic hurricane...,1
1084990,keep water cold decent size fit cup holder loo...,1
2696096,ordered x pack needing total card shipped tota...,2


In [124]:
wv1 = pd.DataFrame()
mymodel1 = pd.DataFrame()

def gentrain(s,model):
    word_list = train(s,model)
    word_list = np.array(word_list)
    word_list = word_list.mean(axis=0)
    return word_list

def train(s,model):
    words=s.split()
    each_word=[]
    for word in words:
        if word in model:
            each_word.append(model[word])
        else:
            each_word.append(np.zeros(300))
    return each_word

wv1['review_body']=df_ternary['review_body'].apply(lambda row: gentrain(row, wv))
mymodel1['review_body']=df_ternary['review_body'].apply(lambda row: gentrain(row, mymodel.wv))

/Users/sidmitta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice.
  import sys


In [125]:
n_wv_trained=normalize(wv1['review_body'])
n_mymodel_trained=normalize(mymodel1['review_body'])

In [126]:
n_wv_trained.shape

(250000, 300)

In [60]:
df_ternary['rating']=df_ternary['rating'].replace(1,0)
df_ternary['rating']=df_ternary['rating'].replace(2,1)
df_ternary['rating']=df_ternary['rating'].replace(3,2)

In [128]:
df_ternary.query("rating==3")

,review_body,rating


In [129]:
avg_my_model_x_train, avg_my_model_x_test, avg_my_model_y_train, avg_my_model_y_test = train_test_split(n_mymodel_trained, df_ternary["rating"],test_size=0.2, random_state=0)
avg_wv_x_train, avg_wv_x_test, avg_wv_y_train, avg_wv_y_test = train_test_split(n_wv_trained, df_ternary["rating"],test_size=0.2, random_state=0)

In [158]:
x_train = torch.tensor(avg_my_model_x_train)
y_train = torch.tensor(avg_my_model_y_train.astype(np.float32).values, dtype=torch.long)
x_test = torch.tensor(avg_my_model_x_test)
y_test = torch.tensor(avg_my_model_y_test.astype(np.float32).values, dtype=torch.long)

In [159]:
x_train.shape

torch.Size([200000, 300])

In [160]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers,)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

In [177]:
class TernaryNet(nn.Module):
    def __init__(self):
        super(TernaryNet, self).__init__()
        # number of hidden nodes in each layer (512)
        hidden_1 = 50
        hidden_2 = 10
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(300, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(hidden_2, 3)
        
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # flatten image input
        #x = x.view(-1, 300)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x.float()))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        #x = self.softmax(x)
        return(x)

# initialize the NN
model = TernaryNet()

In [178]:
train_model()

Epoch: 1 	Training Loss: 0.896904 	Validation Loss: 0.224210
Validation loss decreased (inf --> 0.224210).  Saving model ...
Epoch: 2 	Training Loss: 0.896886 	Validation Loss: 0.224210
Validation loss decreased (0.224210 --> 0.224210).  Saving model ...
Epoch: 3 	Training Loss: 0.896876 	Validation Loss: 0.224210
Validation loss decreased (0.224210 --> 0.224210).  Saving model ...
Epoch: 4 	Training Loss: 0.896881 	Validation Loss: 0.224210
Epoch: 5 	Training Loss: 0.896860 	Validation Loss: 0.224210
Validation loss decreased (0.224210 --> 0.224210).  Saving model ...
Epoch: 6 	Training Loss: 0.896884 	Validation Loss: 0.224210
Validation loss decreased (0.224210 --> 0.224210).  Saving model ...
Epoch: 7 	Training Loss: 0.896873 	Validation Loss: 0.224210
Validation loss decreased (0.224210 --> 0.224210).  Saving model ...
Epoch: 8 	Training Loss: 0.896862 	Validation Loss: 0.224210
Epoch: 9 	Training Loss: 0.896868 	Validation Loss: 0.224210
Epoch: 10 	Training Loss: 0.896859 	Valida

In [163]:
#Custom model accuracy using Feed forward Neural network for ternary classification
test_model()

Accuracy of the network on the test images: 39 %


In [152]:
x_train = torch.tensor(avg_wv_x_train)
y_train = torch.tensor(avg_wv_y_train.astype(np.float32).values, dtype=torch.long)
x_test = torch.tensor(avg_wv_x_test)
y_test = torch.tensor(avg_wv_y_test.astype(np.float32).values, dtype=torch.long)

In [153]:
train_data = []
for i in range(len(x_train)):
    train_data.append([x_train[i], y_train[i]])
test_data = []
for i in range(len(x_test)):
    test_data.append([x_test[i], y_test[i]])
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data)
#indices = list(range(num_train))
indices = [i for i in range(num_train)]
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [154]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers,)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

In [155]:
train_model()

Epoch: 1 	Training Loss: 0.906087 	Validation Loss: 0.226728
Validation loss decreased (inf --> 0.226728).  Saving model ...
Epoch: 2 	Training Loss: 0.906026 	Validation Loss: 0.226728
Validation loss decreased (0.226728 --> 0.226728).  Saving model ...
Epoch: 3 	Training Loss: 0.906079 	Validation Loss: 0.226728
Validation loss decreased (0.226728 --> 0.226728).  Saving model ...
Epoch: 4 	Training Loss: 0.906048 	Validation Loss: 0.226728
Epoch: 5 	Training Loss: 0.906087 	Validation Loss: 0.226728
Epoch: 6 	Training Loss: 0.906043 	Validation Loss: 0.226728
Epoch: 7 	Training Loss: 0.906053 	Validation Loss: 0.226728
Epoch: 8 	Training Loss: 0.906046 	Validation Loss: 0.226728
Epoch: 9 	Training Loss: 0.906056 	Validation Loss: 0.226728
Validation loss decreased (0.226728 --> 0.226728).  Saving model ...
Epoch: 10 	Training Loss: 0.906067 	Validation Loss: 0.226728
Epoch: 11 	Training Loss: 0.906074 	Validation Loss: 0.226728
Epoch: 12 	Training Loss: 0.906060 	Validation Loss: 0.2

In [157]:
#WV model accuracy using feedforward Neural network for ternary classification
test_model()

Accuracy of the network on the test images: 39 %


In [76]:
def genpretrain_ten(s,wv):
    list = pretrain(s,wv)
    #list = np.array(list)
    length = len(list)
    if length<10:
        diff = 10 - length
        newlist = []
        for i in list:
            
            newlist = np.concatenate((newlist, i), axis=None)
        for i in range(diff):
            
            newlist = np.concatenate((newlist,[0.0 for i in range(300)]), axis = None)
        
        
    else:
        newlist=[]
        for i in list[:10]:
            newlist = np.concatenate((newlist, i), axis=None)
    return(newlist)


def normalize_ten(df):
    list= []
    for i in df:
        list.append(np.array(i))
    list = np.array(list)
    return list

def gentrain(s,model):
    word_list = train(s,model)
    word_list = np.array(word_list)
    word_list = word_list.mean(axis=0)
    return word_list

def train(s,model):
    words=s.split()
    each_word=[]
    for word in words:
        if word in model:
            each_word.append(model[word])
        else:
            each_word.append(np.zeros(300))
    return each_word


def pretrain(s,wv):
    j = s.split()
    list= []
    for word in j :
        if word not in wv:
            list.append([0 for i in range(300)])
        else:
            list.append(wv[word])
    
    return list

In [77]:
wv_10 = pd.DataFrame()
mymodel_10 = pd.DataFrame()

wv_10['review_body']=df_ternary['review_body'].apply(lambda row: genpretrain_ten(row, wv))
mymodel_10['review_body']=df_ternary['review_body'].apply(lambda row: genpretrain_ten(row, mymodel.wv))

In [79]:
n_wv_trained=normalize_ten(wv_10['review_body'])
n_mymodel_trained=normalize_ten(mymodel_10['review_body'])

In [81]:
avg_my_model_x_train, avg_my_model_x_test, avg_my_model_y_train, avg_my_model_y_test = train_test_split(n_mymodel_trained, df_ternary["rating"],test_size=0.2, random_state=0)


In [ ]:
avg_wv_x_train, avg_wv_x_test, avg_wv_y_train, avg_wv_y_test = train_test_split(n_wv_trained, df_ternary["rating"],test_size=0.2, random_state=0)

In [87]:
x_train = torch.tensor(avg_wv_x_train)
y_train = torch.tensor(avg_wv_y_train.astype(np.float32).values, dtype=torch.long)
x_test = torch.tensor(avg_wv_x_test)
y_test = torch.tensor(avg_wv_y_test.astype(np.float32).values, dtype=torch.long)

train_data = []
for i in range(len(x_train)):
    train_data.append([x_train[i], y_train[i]])
test_data = []
for i in range(len(x_test)):
    test_data.append([x_test[i], y_test[i]])
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data)
#indices = list(range(num_train))
indices = [i for i in range(num_train)]
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers,)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

In [88]:
class TernaryNet_10(nn.Module):
    def __init__(self):
        super(TernaryNet_10, self).__init__()
        # number of hidden nodes in each layer (512)
        hidden_1 = 50
        hidden_2 = 10
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(3000, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(hidden_2, 3)
        
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # flatten image input
        #x = x.view(-1, 300)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x.float()))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        #x = self.softmax(x)
        return(x)

In [89]:
model = TernaryNet_10()

In [90]:
train_model()

Epoch: 1 	Training Loss: 0.881601 	Validation Loss: 0.220370
Validation loss decreased (inf --> 0.220370).  Saving model ...
Epoch: 2 	Training Loss: 0.881581 	Validation Loss: 0.220370
Validation loss decreased (0.220370 --> 0.220370).  Saving model ...
Epoch: 3 	Training Loss: 0.881476 	Validation Loss: 0.220370
Epoch: 4 	Training Loss: 0.881517 	Validation Loss: 0.220370
Validation loss decreased (0.220370 --> 0.220370).  Saving model ...
Epoch: 5 	Training Loss: 0.881577 	Validation Loss: 0.220370
Validation loss decreased (0.220370 --> 0.220370).  Saving model ...
Epoch: 6 	Training Loss: 0.881553 	Validation Loss: 0.220370
Epoch: 7 	Training Loss: 0.881568 	Validation Loss: 0.220370
Epoch: 8 	Training Loss: 0.881542 	Validation Loss: 0.220370
Validation loss decreased (0.220370 --> 0.220370).  Saving model ...
Epoch: 9 	Training Loss: 0.881555 	Validation Loss: 0.220370
Epoch: 10 	Training Loss: 0.881468 	Validation Loss: 0.220370
Epoch: 11 	Training Loss: 0.881565 	Validation Lo

In [91]:
#WV model ternary classification accuracy for concatenated vectors using Feedforward neural network
test_model()

Accuracy of the network on the test images: 39 %


In [92]:
def prepareTrainandTest_ten(train_x,train_y,test_x,test_y,wv):

    x_train = torch.tensor(train_x)
    y_train = torch.tensor(train_y.astype(np.float32).values, dtype=torch.long)
    x_test = torch.tensor(test_x)
    y_test = torch.tensor(test_y.astype(np.float32).values, dtype=torch.long)
    train_data = []
    for i in range(len(x_train)):
        train_data.append([x_train[i], y_train[i]])
    test_data = []
    for i in range(len(x_test)):
        test_data.append([x_test[i], y_test[i]])
    # number of subprocesses to use for data loading
    num_workers = 0
    # how many samples per batch to load
    batch_size = 20
    # percentage of training set to use as validation
    valid_size = 0.2

    # obtain training indices that will be used for validation
    num_train = len(train_data)
    #indices = list(range(num_train))
    indices = [i for i in range(num_train)]
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    # define samplers for obtaining training and validation batches
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
        sampler=train_sampler, num_workers=num_workers,)
    valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
        sampler=valid_sampler, num_workers=num_workers)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
        num_workers=num_workers)
    
    
    return (train_data, test_data, train_loader, valid_loader, test_loader)

In [93]:
train_data, test_data, train_loader, valid_loader, test_loader = prepareTrainandTest_ten(avg_my_model_x_train,avg_my_model_y_train,avg_my_model_x_test,avg_my_model_y_test,mymodel.wv)


In [95]:
train_model()

Epoch: 1 	Training Loss: 0.906275 	Validation Loss: 0.225647
Validation loss decreased (inf --> 0.225647).  Saving model ...
Epoch: 2 	Training Loss: 0.906103 	Validation Loss: 0.225647
Epoch: 3 	Training Loss: 0.906253 	Validation Loss: 0.225647
Validation loss decreased (0.225647 --> 0.225647).  Saving model ...
Epoch: 4 	Training Loss: 0.906247 	Validation Loss: 0.225647
Epoch: 5 	Training Loss: 0.906285 	Validation Loss: 0.225647
Epoch: 6 	Training Loss: 0.906019 	Validation Loss: 0.225647
Epoch: 7 	Training Loss: 0.906270 	Validation Loss: 0.225647
Validation loss decreased (0.225647 --> 0.225647).  Saving model ...
Epoch: 8 	Training Loss: 0.906395 	Validation Loss: 0.225647
Epoch: 9 	Training Loss: 0.906126 	Validation Loss: 0.225647
Epoch: 10 	Training Loss: 0.906000 	Validation Loss: 0.225647
Epoch: 11 	Training Loss: 0.906247 	Validation Loss: 0.225647
Epoch: 12 	Training Loss: 0.906024 	Validation Loss: 0.225647
Epoch: 13 	Training Loss: 0.906137 	Validation Loss: 0.225647
E

In [96]:
#Custom model ternary classification accuracy for concatenated vectors using Feedforward neural network
test_model()

Accuracy of the network on the test images: 27 %


In [98]:
df_binary_10 = pd.DataFrame()
df_binary_10 = df_binary_2.copy()

In [100]:
df_binary_10["rating"]=df_binary_2["rating"].apply(lambda x: 0 if x == 1 else x)


In [102]:
df_binary_10["rating"]=df_binary_10["rating"].apply(lambda x: 1 if x == 2 else x)


In [104]:
wv_10_binary = pd.DataFrame()
mymodel_10_binary = pd.DataFrame()

wv_10_binary['review_body']=df_binary_10['review_body'].apply(lambda row: genpretrain_ten(row, wv))
mymodel_10_binary['review_body']=df_binary_10['review_body'].apply(lambda row: genpretrain_ten(row, mymodel.wv))

In [105]:
n_wv_trained=normalize_ten(wv_10_binary['review_body'])
n_mymodel_trained=normalize_ten(mymodel_10_binary['review_body'])

In [106]:
#from sklearn.model_selection import train_test_split

avg_my_model_x_train, avg_my_model_x_test, avg_my_model_y_train, avg_my_model_y_test = train_test_split(n_mymodel_trained, df_binary_10["rating"],test_size=0.2, random_state=0)

In [107]:
avg_wv_x_train, avg_wv_x_test, avg_wv_y_train, avg_wv_y_test = train_test_split(n_wv_trained, df_binary_10["rating"],test_size=0.2, random_state=0)

In [108]:
train_data, test_data, train_loader, valid_loader, test_loader = prepareTrainandTest_ten(avg_wv_x_train,avg_wv_y_train,avg_wv_x_test,avg_wv_y_test,wv)


In [113]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (512)
        hidden_1 = 50
        hidden_2 = 10
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(3000, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(hidden_2, 2)
        
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # flatten image input
        #x = x.view(-1, 300)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x.float()))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        #x = self.softmax(x)
        return(F.log_softmax(x,dim=1))

# initialize the NN
model = Net()

In [115]:
train_model()

Epoch: 1 	Training Loss: 0.555174 	Validation Loss: 0.138792
Validation loss decreased (inf --> 0.138792).  Saving model ...
Epoch: 2 	Training Loss: 0.555142 	Validation Loss: 0.138792
Validation loss decreased (0.138792 --> 0.138792).  Saving model ...
Epoch: 3 	Training Loss: 0.555162 	Validation Loss: 0.138792
Epoch: 4 	Training Loss: 0.555144 	Validation Loss: 0.138792
Epoch: 5 	Training Loss: 0.555170 	Validation Loss: 0.138792
Epoch: 6 	Training Loss: 0.555130 	Validation Loss: 0.138792
Epoch: 7 	Training Loss: 0.555171 	Validation Loss: 0.138792
Epoch: 8 	Training Loss: 0.555151 	Validation Loss: 0.138792
Epoch: 9 	Training Loss: 0.555138 	Validation Loss: 0.138792
Epoch: 10 	Training Loss: 0.555154 	Validation Loss: 0.138792
Epoch: 11 	Training Loss: 0.555153 	Validation Loss: 0.138792
Validation loss decreased (0.138792 --> 0.138792).  Saving model ...
Epoch: 12 	Training Loss: 0.555133 	Validation Loss: 0.138792
Epoch: 13 	Training Loss: 0.555149 	Validation Loss: 0.138792
E

In [116]:
#WV model binary classification accuracy for concatenated vectors using Feedforward neural network
test_model()

Accuracy of the network on the test images: 50 %


In [118]:
train_data, test_data, train_loader, valid_loader, test_loader = prepareTrainandTest_ten(avg_my_model_x_train, avg_my_model_y_train, avg_my_model_x_test, avg_my_model_y_test,mymodel.wv)


In [119]:
model = Net()

In [120]:
train_model()

Epoch: 1 	Training Loss: 0.574479 	Validation Loss: 0.142603
Validation loss decreased (inf --> 0.142603).  Saving model ...
Epoch: 2 	Training Loss: 0.574554 	Validation Loss: 0.142603
Epoch: 3 	Training Loss: 0.574687 	Validation Loss: 0.142603
Epoch: 4 	Training Loss: 0.574609 	Validation Loss: 0.142603
Epoch: 5 	Training Loss: 0.574656 	Validation Loss: 0.142603
Epoch: 6 	Training Loss: 0.574843 	Validation Loss: 0.142603
Epoch: 7 	Training Loss: 0.574786 	Validation Loss: 0.142603
Validation loss decreased (0.142603 --> 0.142603).  Saving model ...
Epoch: 8 	Training Loss: 0.574532 	Validation Loss: 0.142603
Epoch: 9 	Training Loss: 0.574749 	Validation Loss: 0.142603
Epoch: 10 	Training Loss: 0.574592 	Validation Loss: 0.142603
Epoch: 11 	Training Loss: 0.574588 	Validation Loss: 0.142603
Epoch: 12 	Training Loss: 0.574543 	Validation Loss: 0.142603
Epoch: 13 	Training Loss: 0.574550 	Validation Loss: 0.142603
Epoch: 14 	Training Loss: 0.574672 	Validation Loss: 0.142603
Epoch: 1

In [121]:
#Custom model binary classification accuracy for concatenated vectors using Feedforward neural network
test_model()

Accuracy of the network on the test images: 50 %


In [207]:
class RNN_Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNN_Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=False)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [ ]:
device = torch.device("cpu")

In [ ]:
# Instantiate the model with hyperparameters
model = RNN_Model(input_size=300, output_size=2, hidden_dim=50, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
lr=0.001

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [ ]:
def genpretrain_rnn(s,wv):
    list = pretrain_rnn(s,wv)
    list = np.array(list)
    #list = list.astype(np.float64)
    #list = list.mean(axis=0)
    return list

def pretrain_rnn(s,wv):
    j = s.split()
    if len(j)>20:
        j = j[:20]
        list = []
        for word in j :
            if word not in wv:
                list.append([0 for i in range(300)])
            else:
                list.append(wv[word])
    else:
        diff = 20 - len(j)
        list = []
        for word in j :
            if word not in wv:
                list.append([0 for i in range(300)])
            else:
                list.append(wv[word])
                
        for i in range(diff):
            list.append([0 for i in range(300)])
    
    return list

def normalize_rnn(df):
    rlist= []
    for i in df:
        list =[]
        for j in i:
            list.append(np.array(j))
        rlist.append(np.array(list))
    rlist = np.array(rlist)
    return rlist

In [ ]:
wv_rnn_binary = pd.DataFrame()
mymodel_rnn_binary = pd.DataFrame()

wv_rnn_binary['review_body']=df_binary['review_body'].apply(lambda row: genpretrain_rnn(row, wv))
mymodel_rnn_binary['review_body']=df_binary['review_body'].apply(lambda row: genpretrain_rnn(row, mymodel.wv))

In [ ]:
n_wv_trained=normalize_rnn(wv_rnn_binary['review_body'])
n_mymodel_trained=normalize_rnn(mymodel_rnn_binary['review_body'])

In [ ]:
avg_my_model_x_train, avg_my_model_x_test, avg_my_model_y_train, avg_my_model_y_test = train_test_split(n_mymodel_trained, df_ternary["rating"],test_size=0.2, random_state=0)
avg_wv_x_train, avg_wv_x_test, avg_wv_y_train, avg_wv_y_test = train_test_split(n_wv_trained, df_ternary["rating"],test_size=0.2, random_state=0)

In [ ]:
train_data, test_data, train_loader, valid_loader, test_loader = prepareTrainandTest_ten(avg_my_model_x_train, avg_my_model_y_train, avg_my_model_x_test, avg_my_model_y_test,mymodel.wv)


In [ ]:
train_model()

In [210]:
test_model()

Accuracy of the network on test images: 64%


In [ ]:
train_data, test_data, train_loader, valid_loader, test_loader = prepareTrainandTest_ten(avg_wv_x_train,avg_wv_y_train,avg_wv_x_test,avg_wv_y_test,wv)

In [ ]:
train_model()

In [212]:
test_model()

Accuracy of the network on test images: 62%
